----------------------------------- ----------------------------------- ----------------------------------- 
----------------------------------- ESPACIO PARA BANNER DE LA MAESTRIA -----------------------------------
----------------------------------- ----------------------------------- ----------------------------------- 

# Análisis de Componentes Principales. Detalles sobre su Implementación.

En este *cuaderno* continuamos estudiando el análisis de componentes principales. El objetivo es que aprenda que detalles sobre su implementación.

**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podés modificarlo libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.

## Escalar las variables

En los ejemplos del cuaderno de *Fundamentos Teóricos* utilizamos datos que estaban estandarizados, es decir, recentrados para tener media cero y escalados para tener varianza uno.  Si bien desde un punto estrictamente matemático no hay nada intrínsecamente incorrecto, hacer combinaciones lineales de variables con diferentes unidades de medida. Sin embargo cuando usamos PCA buscamos maximizar varianza y la varianza se ve afectada por las unidades de medida. Esto implica quelos componentes principales basados en la matriz de covarianza $S$ va a cambiar si las unidades de medida de una o mas variables cmabian. Para que esto no suceda es práctica habitual  estandarizar las variables. Es decir cada valor de $X$ es recentrado y dividio por la desviación estandard:

\begin{align}
z_{ij} = \frac{x_{ij}-\bar{x_j}}{s_j}
\end{align}

donde $\bar{x_j}$ y ${s_j}$ son la media y el desvio estandard de la columna $j$. Entonces, la matriz de datos iniciales $X$ es remplazado por la matriz de datos estandarizados $Z$. Notemos tambien que al estandarizar la matriz de datos, la matriz de covarianca $S$ es simplemente la matriz de correlacion de los datos original. Esto a veces en la literatura se lo conoce como la matriz de correlación de PCA.

Esto contrasta con otras técnicas de aprendizaje supervizado como es la regresión lineal donde esclar las variables no tiene ningun efecto. Si multiplicamos una variable por $\frac{1}{\alpha}$, los coeficentes estaran multiplicados por $\alpha$ no teniendo un efecto en el modelo obtenido.

Para ilustrar como estandarizar en `Python` vamos a utilizar los datos de la  Encuesta Nacional de Presupuestos de los Hogares (ENPH) de Colombia, realizada por el DANE en 2017. Los datos contienen información sobre el gastos promedio en distintos rubros para 38 ciudades colombianas.

In [32]:
#Cargamos las librerias a utilizar
import pandas as pd
import numpy as np

# Cargamos y visualizamos la primeras observaciones de los datos
X = pd.read_csv('Data/gasto_col_2017.csv')
X = X.set_index("Ciudad")
X.head()


,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,"Alojamiento, agua, electricidad, gas y otros combustibles","Muebles, artículos para el hogar y para la conservación ordinaria del hogar",Salud,Transporte,Información y comunicación,Recreación y cultura,Educación,Restaurantes y hoteles,Bienes y servicios diversos
Ciudad,,,,,,,,,,,,
Arauca,76.532496,11.983778,21.065229,158.952746,15.040745,11.793285,20.656260,20.404561,11.523317,24.240347,44.130546,55.110519
Armenia,104.227192,18.610392,39.651621,174.252141,26.524739,14.846364,71.426343,30.486730,28.148628,58.902919,62.735795,145.317221
Barrancabermeja,73.495641,23.511551,25.204020,218.377925,16.637673,8.367648,51.842837,25.653363,12.853022,68.650880,42.090619,58.657609
Barranquilla,85.059421,16.028608,23.798484,204.792777,20.162867,7.785154,61.043240,24.506067,12.261522,58.348581,54.051667,72.252969
Bogotá,103.561150,21.839434,42.814236,295.817640,34.168082,27.459289,111.870806,40.657457,36.484350,134.933249,94.643529,236.887545


In [33]:
# Estandarizamos los datos
mu = X.mean()
sigma = X.std()
Z = (X - mu)/sigma
Z.head()

,Alimentos y bebidas no alcohólicas,Bebidas alcohólicas y tabaco,Prendas de vestir y calzado,"Alojamiento, agua, electricidad, gas y otros combustibles","Muebles, artículos para el hogar y para la conservación ordinaria del hogar",Salud,Transporte,Información y comunicación,Recreación y cultura,Educación,Restaurantes y hoteles,Bienes y servicios diversos
Ciudad,,,,,,,,,,,,
Arauca,-0.893243,-1.491704,-1.212847,-0.187501,-0.765951,-0.125062,-1.344088,-0.438465,-0.757958,-1.028321,-0.413888,-0.778491
Armenia,1.411143,-0.317263,1.339782,0.137351,1.208855,0.434314,0.691669,0.965570,1.631818,0.528711,0.552798,1.380602
Barrancabermeja,-1.145930,0.551373,-0.644432,1.074275,-0.491341,-0.752697,-0.093582,0.292479,-0.566821,0.966586,-0.519878,-0.693592
Barranquilla,-0.183745,-0.774835,-0.837466,0.785821,0.114857,-0.859420,0.275332,0.132708,-0.651846,0.503810,0.101591,-0.368188
Bogotá,1.355724,0.255023,1.774131,2.718554,2.523217,2.745217,2.313395,2.381936,2.830021,3.943969,2.210651,3.572333


Lo invito a comparar estos resultados con los de los datos utilizados en el *cuaderno* de *Fundamentos Teóricos*.

## Proporción de Varianza Explicada


Una propiedad muy útil del procedimiento del cálculo de componentes principales es que la variabilidad total de $X$ es la suma de los $k$ eigenvalues de $V(X)=S$. Para demostrarlo supongamos que $\lambda_1,\dots,\lambda_k$ son los eigenvalues de $V(X)=S$, ordenados de mayor a menor, y que $p_1 , \dots , p_K$ son los eigenvectores correspondientes. Adicionalmente llamemos $P$ a la matriz cuyas columnas son estos eigenvectores.

Supongamos tambien que $f_j= X \delta'_j$ es el $j-ésimo$ compontente principal, entonces

\begin{align}
V (f_j ) &= \delta_j S \delta'_j  \\\
         &= p_j P\Lambda P' p'_j  \\\
         &= \lambda_j
\end{align}

Donde la segunda igualdad se desprende de la descomposición espectral. Recordemos que la descomposición espectral establece que si $A_{m\times m}$ entonces $A=P\Lambda P'$ donde $\Lambda=diag(\lambda_1,\dots,\lambda_k)$ y $P$ es la matriz cuyas columnas son los eigenvectors correspondientes.

Este resultado nos dices que la varianza del j− esimo componente principal es el j− esimo autovalor ordenado de S.

Usando este resultado podemos ver que la varianza total de $X$ va a ser la suma de las varianza de las variables $x_j$ ya que:

\begin{align}
tr(\Sigma) = tr(P \Lambda P')= tr(PP' \Lambda ) = \sum_{j=1}^k \lambda_j= \sum_{j=1}^k V(f_j)
\end{align}


Este resultado nos permite preguntarnos: ¿Cuánta información perdimos por proyectar los datos en unos cuantos componentes principales? Esto es, ¿Cuánto de la varianza esta contenida en los primeros componentes principales? Usando la ecuación anterior podemos ver la propoción de varianza contenida en cada componente principal:

\begin{align}
\frac{\lambda_k}{\sum_{j=1}^k \lambda_j}
\end{align}

Usando el ejemplo del *cuaderno* de *Fundamentos Teóricos*, podemos calcular la propoción de varianza contenida en cada componente principal:

In [24]:
gasto = pd.read_csv('Data/gasto_col_2017_norm.csv')
gasto = gasto.set_index("Ciudad")
S = gasto.cov()
eig_val, eig_vec = np.linalg.eig(S)
eig_val = np.sort(eig_val)[::-1] #eigenvalues ordenados de mayor a menor
eig_val

array([2.25116235, 0.4617445 , 0.28709315])

In [25]:
eig_val/sum(eig_val)

array([0.75038745, 0.15391483, 0.09569772])

Podemos ver entonces que en este ejemplo:

- El primer componente principal explica el 75% del total de la varianza.

- El segundo componente principal explica el 15.4% del total de la varianza.

- El tercer componente principal explica el 9.6% del total de la varianza.

Finalmente notemos que la suma da 100%. Lo invito a replicar y comparar estos resultados con los datos utilizados en el punto anterior.

## ¿Cuántos componentes principales debemos seleccionar?


En el *cuaderno* de *Fundamentos Teóricos* mencionamos que una matriz $X$ de dimensión $n \times k$ tiene en general  $min(n-1,k)$, componentes principales distintos.  En la práctica generalmente no estamos interesados en todos los componentes, sino más bien quedarnos con los primeros que nos permitan vizualizar o interpretar datos. En efecto, nos gustaría quedarnos con el minoimo numero que nos permita una buena comprención de los datos. Por ejemplo, en el ejemplo anterior los dos primeros componentes explican mas del 90% del total de la varianza. Es decir en este caso con dos componentes logramos una buena representación de los datos.

La pregunta natural que surge aqui es si hay una forma establecida de determinar el número de componentes principales a utilizar. Desafortunadamente, no existe una forma objetiva aceptada en la literatura de responderla. Sin embargo, hay tres enfoques simples que pueden servir de guía para decidir el número de componentes principales relevantes.

Estos son:

- Examen visual de un gráfico de sedimentación (scree plot).
- Proporción de varianza explicada.
- Regla de Kaiser.


### Examen visual de un gráfico de sedimentación (scree plot)

Un enfoque ampliamente utilizado consiste en elegir el número de componentes principales a utilizar examinando un diagrama de sedimentación o scree plot por su nombre en inglés. La gráfica abajo muestra un scree plot, el panel A resume la proporción de varianza explicada por cada componente principal y en el panel B como suma acumuladas.

![](aux/plot_S1_LSC2_1.png)

Examinando visualmente la gráfica buscamos un punto donde la proporción de varianza explicada para cada componente principal cae. Esta caida se conoce a menudo como  sedimento o codo del gráfico. La idea de sedimento proviene de comparar la linea en el panel A a la ladera de una montaña y buscar el punto donde si cayesen rocas estas comenzarian a pararse o sedimentarse (de alli el nombre). 

Por ejemplo, en base al grafico anterior podemos decir que luego del tercer componente principal hay un codo o un area de sedimentación. Siguiendo entonces este criterio nos quedariamos con 3 componentes principales para representar nuestro conjunto de datos, lo que explica el 83% de la varianza en el conjunto de datos. Notemos que después del tercer componente los componentes explican menos del 5% de los datos.

Sin embargo, este tipo de análisis visual es inherentemente *ad hoc*. Desafortunadamente, no existe una forma objetiva bien aceptada de decidir cuántos componentes principales son suficientes. 

### Proporción de varianza explicada

El análisis visual anterior es inherentemente *ad hoc*. Otro enfoque a menudo utilizado en la practica es imponer un umbral a priori y elegir los componentes principales en base a esta. Por ejemplo podríamos definr un umbral del 90% lo que en el ejemplo anterior nos resultaria en 5 componentes principales. Mientras que si fuese el 70% tendriamos 2 componentes principales.

El umbral a definir dependera de la aplicación, el contexto, y el conjunto de los datos. Tipicamente se utilizan umbrales entre el 70% y el 90%.


### Criterio de Kaiser

El criterio de Kaise es otro enfoque ampliamente utilizado para evaluar el número máximo de componentes principales. Este sugiere que solo se retienen los componentes principales cuyos eigenvalues exceden 1. La idea es que se retengan aquellas componentes cuyos eigenvalues sean superiores a la media de los eigenvalues:

\begin{align}
\lambda_h> \frac{\sum_j^k \lambda_j}{k}
\end{align}

dado que los datos estan standarizados tenemos que $\sum_j^k \lambda_j=k$ por lo que es equivalente a buscar los eigenvalues mayores a uno.

### Consideraciones  finales sobre el número de componentes a elegir

Es importante reiterar de que no exista una forma objetiva aceptada para seleccionar el número de componentes a utilizar. De hecho la cuestión esta intrínsecamente mal definida y todo va a depender del área y los datos donde se utilizan.

En la práctica de  aprendizaje no supervizado, tendemos a examinar los primero componentes principalesen busqueda de patrones interesantes en los datos. Si no se encuentran, es poco probable que el resto de los componentes de interés. Por el contrario, si estos son interesantes se continua hasta que no haya mas patrones. Esto es un enfoque completamente subjetivo y refleja el hecho de que utilizamos PCA como una herramientas de análisis supervizado.

Por otro lado, si utilizamos PCA como insumos en aprendizaje supervisado, por ejemplo los utilizamos como features en una regresión lineal, entonces existe una forma simple y objetiva de determinar el número de componentes principales usar. En este caso se utiliza validación cruzada para seleccionar el número optimo. 

El contraste en dificultades de seleccionar el número de componentes para aprendizaje  no supervizados y supervisados reflejan el hecho de que el análisis supervisado tiende a estar más claramente definido y evaluado que los no supervisados.

## Unicidad de los componentes principales


Antes de finalizar es necesitamos hacer la advertencia de que los "loadings" de los componentes principales ($\delta_{ij}=\delta_{i1},\dots, \delta_{ik}$) son únicos excepto por un cambio de signo. Esto implica que dependiendo la implementación podemos obtener en dos librerias resultados distintos. Los "loadings" seran los mismos pero los signos pueden diferir. Los signos pueden diferir porque cada peso especifica una dirección en el espacio k-dimensional y el cambio de signo no tiene efecto sobre la dirección.  

## Referencias


-   Ahumada, H. A., Gabrielli, M. F., Herrera Gomez, M. H., & Sosa
    Escudero, W. (2018). Una nueva econometría: Automatización, big
    data, econometría espacial y estructural.

-   DANE (29 de Septiembre de 2020). Encuesta nacional de presupuestos
    de los hogares (ENPH). Anexos: 32 ciudades y 6 ciudades intermedias.
    <https://www.dane.gov.co/files/investigaciones/boletines/enph/ciudades-enph-2017.xls>

-   Deisenroth, M. P., Faisal, A. A., & Ong, C. S. (2020). Mathematics
    for machine learning. Cambridge University Press.
    
-   Hartmann, K., Krois, J., Waske, B. (2018): E-Learning Project SOGA: Statistics and Geospatial Data Analysis. Department of Earth Sciences, Freie Universitaet Berlin.    

-   James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). An
    introduction to statistical learning (Vol. 112, p. 18). New York:
    springer.

-   Jolliffe, I. T., & Cadima, J. (2016). Principal component analysis: a review and recent developments. Philosophical transactions. Series A, Mathematical, physical, and engineering sciences, 374(2065), 20150202. https://doi.org/10.1098/rsta.2015.0202

-   Murphy, K. P. (2012). Machine learning: a probabilistic perspective.
    MIT press.

-   Peña, D. (2002). Análisis de datos multivariantes (Vol. 24). Madrid:
    McGraw-hill.


